In [17]:
import numpy as np
from scipy.sparse import diags, kron, eye, linalg
from scipy.sparse.linalg import spsolve, splu
import matplotlib.pyplot as plt
from matplotlib import animation, cm
from matplotlib.animation import FuncAnimation
%matplotlib qt

# Coefficients
Lx = 12
Ly = 5
T_ext = 25

# A
N = 480
h = Lx / N
M = int(Ly / h)


In [18]:
def F_func(x, y):
    return 100 * np.exp(-1/2 * (x - 4)**2 - 4 * (y - 1)**2)

x = np.arange(h, Lx, h)
y = np.arange(h, Ly, h)
X, Y = np.meshgrid(x, y)
F = F_func(X, Y).T

# Create the sparse matrices Sx and Sy
Sx = (1 / h**2) * diags([1, -2, 1], [-1, 0, 1], shape=(N-1, N-1),format='csc')
Sy = (1 / h**2) * diags([1, -2, 1], [-1, 0, 1], shape=(M-1, M-1), format='csc')

# Apply boundary conditions
Sx[0, 0] = -2 / (3 * h**2)
Sx[0, 1] = 2 / (3 * h**2)
Sx[-1, -1] = -2 / (3 * h**2)
Sx[-1, -2] = 2 / (3 * h**2)

Sy[-1, -1] = -2 / (3 * h**2)
Sy[-1, -2] = 2 / (3 * h**2)

# Create the operator A using Kronecker products
A = kron(eye(M-1), Sx, format='csc') + kron(Sy, eye(N-1), format='csc')

F[:, 0] = F[:, 0] + T_ext / h**2
f = F.reshape(((N-1) * (M-1), 1), order='F')

dt = 0.1
T = 40

u0 = T_ext * np.ones(f.shape)

# Crank Nicolson
tau = np.arange(0, T+dt, dt)
saved_u = np.zeros((len(u0), len(tau)))
saved_u[:, 0] = u0[:, 0]

uk = u0

I = eye(len(u0), format='csc')
LHS = splu(I - 1/2 * dt * A)
for i in range(len(tau)-1):
    RHS = (I + 1/2 * dt * A).dot(uk) + dt * f
    u_new = LHS.solve(RHS)
    saved_u[:, i+1] = u_new[:,0]
    uk = u_new


# Reshape to 3D array and apply boundaries
u = saved_u.reshape(N - 1, M - 1, int(T/dt)+1,order='F')
u_y0 = T_ext * np.ones((N - 1,int(T/dt)+1))
u_M = 1/3 * (4 * u[:, -1,:] - u[:, -2,:])
u_y0 = u_y0[:, np.newaxis, :]
u_M = u_M[:, np.newaxis, :]
u = np.concatenate((u_y0,u,u_M),axis=1)
u_N = 1/3 * (4 * u[-1, :,:] - u[-2, :,:])
u_x0 = 1/3 * (4 * u[0, :,:] - u[1, :,:])
u_N = u_N[np.newaxis,:, :]
u_x0 = u_x0[np.newaxis,:, :]
u = np.concatenate((u_x0,u,u_N),axis=0)

x, y = np.meshgrid(np.arange(0, Lx + h, h), np.arange(0, Ly + h, h))
x, y = x.T, y.T  # Transpose x and y to match the shape of u

In [23]:

# Create a figure and 3D axis for the animation
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
# Function to update the 3D plot for each frame
def update(frame):
    ax.clear()
    ax.plot_surface(x, y, u[:,:,frame], cmap=cm.jet)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("Temperature")
    ax.set_zlim(25,70)
    ax.set_xlim(Lx,0)
    ax.set_ylim(0,Ly)
    ax.set_title(f'Time {np.round(frame*dt+dt,2)} seconds')

ani = FuncAnimation(fig, update, frames=range(int(T / dt)), repeat=False, interval = 1)
ani.save('animation.gif', fps=30)


MovieWriter ffmpeg unavailable; using Pillow instead.


KeyboardInterrupt: 

In [22]:
frames = [0, 1, 4, 12, 22, 40]

for i in frames:

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.clear()
    ax.plot_surface(x, y, u[:,:,int(i/dt)], cmap=cm.jet, rstride = 10, cstride = 10)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("Temperature")
    ax.set_zlim(25,70)
    ax.set_xlim(Lx,0)
    ax.set_ylim(0,Ly)
    ax.set_title(f'Time {i} seconds')

    

In [5]:
# C
u_62 = u[int(6/h)][int(2/h)]

print(f"u(6,2,40) = {u_62[-1]:.4f}")

plt.plot(tau,u_62)


u(6,2,40) = 46.9636
